In [5]:
# Imports and helpers

import pydicom
from pydicom.dataset import Dataset
from pydicom.datadict import keyword_for_tag, dictionary_description

def safe_keyword(tag) -> str:
    try:
        return keyword_for_tag(tag) or ""
    except Exception:
        return ""

def safe_description(tag) -> str:
    try:
        return dictionary_description(tag) or ""
    except Exception:
        return ""  # or "Unknown"

def element_header(elem):
    tag = elem.tag
    return {
        "tag": f"({tag.group:04X},{tag.element:04X})",
        "keyword": safe_keyword(tag),
        "name": safe_description(tag),
        "vr": elem.VR,
    }

def dataset_to_dict(ds, depth=0, max_depth=20):
    if depth > max_depth:
        return {"_note": f"Max depth {max_depth} reached"}

    out = []
    for elem in ds:
        # Skip Pixel Data
        if elem.tag.group == 0x7FE0:
            continue

        entry = element_header(elem)

        if elem.VR == "SQ":
            items = []
            for item in (elem.value or []):
                if isinstance(item, Dataset):
                    items.append(dataset_to_dict(item, depth + 1, max_depth))
                else:
                    items.append(to_jsonable(item))
            entry["value"] = items
        else:
            val = elem.value
            if isinstance(val, (list, tuple)):
                entry["value"] = [to_jsonable(v) for v in val]
            else:
                entry["value"] = to_jsonable(val)

        out.append(entry)
    return out

print("pydicom version:", pydicom.__version__)

pydicom version: 3.0.1


In [6]:
# Choose the DICOM file to convert

# 🔧 Update this line to pick the file you want to convert:
dicom_path = "/workspaces/DICOM-reader-for-Copilot/DCMTPS_Calculated(VC04 RESIDENT).dcm"   # e.g., "dicoms/RP1234.dcm" or "RTDOSE.dcm"

# Sanity check: list files in the current folder and verify path exists
print("Current working dir:", os.getcwd())
print("Top-level files:", os.listdir("."))

if not os.path.exists(dicom_path):
    raise FileNotFoundError(f"Could not find: {dicom_path}")
else:
    print("Found DICOM:", dicom_path)

Current working dir: /workspaces/DICOM-reader-for-Copilot
Top-level files: ['.git', 'dicom_to_json.ipynb', '.DICOMtoJSON', 'DCMTPS_Calculated(VC04 RESIDENT).dcm', 'README.md', 'requirement.txt', 'DCMRT_Plan(VC04 RESIDENT).dcm']
Found DICOM: /workspaces/DICOM-reader-for-Copilot/DCMTPS_Calculated(VC04 RESIDENT).dcm


In [7]:
# Read and inspect the dataset (quick peek)

ds = pydicom.dcmread(dicom_path, stop_before_pixels=True)

# Show a quick peek of a few elements
for i, elem in enumerate(ds):
    if i >= 10:
        break
    print(f"{elem.tag} {elem.VR} {keyword_for_tag(elem.tag) or ''} = {str(elem.value)[:120]}")

(0008,0005) CS SpecificCharacterSet = ISO_IR 100
(0008,0008) CS ImageType = ['ORIGINAL', 'PRIMARY', 'DOSE']
(0008,0012) DA InstanceCreationDate = 20260219
(0008,0013) TM InstanceCreationTime = 204134
(0008,0016) UI SOPClassUID = 1.2.840.10008.5.1.4.1.1.481.2
(0008,0018) UI SOPInstanceUID = 1.3.6.1.4.1.2452.6.340338333.1266141038.4258741673.1065990839
(0008,0020) DA StudyDate = 
(0008,0023) DA ContentDate = 20260219
(0008,0030) TM StudyTime = 
(0008,0033) TM ContentTime = 204134


In [8]:
# Convert to JSON (full recursive metadata)

metadata = dataset_to_dict(ds)

json_path = os.path.splitext(dicom_path)[0] + ".json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print("Wrote:", json_path, "| Size:", os.path.getsize(json_path), "bytes")

Wrote: /workspaces/DICOM-reader-for-Copilot/DCMTPS_Calculated(VC04 RESIDENT).json | Size: 41949 bytes


In [9]:
# (Optional) Display a small slice inline to verify

def find_first(keyword_name: str):
    """Find the first element with a given keyword in the JSON structure."""
    for item in metadata:
        if item.get("keyword") == keyword_name:
            return item
    return None

for kw in ["FractionGroupSequence", "BeamSequence", "ControlPointSequence", "DoseReferenceSequence",
           "DoseUnits", "DoseType", "GridFrameOffsetVector", "PixelSpacing", "DoseGridScaling"]:
    hit = find_first(kw)
    if hit:
        print(f"=== Found {kw} ===")
        # Print a condensed view
        print(json.dumps(hit, indent=2)[:1500] + "\n...(truncated)...\n")

=== Found DoseUnits ===
{
  "tag": "(3004,0002)",
  "keyword": "DoseUnits",
  "name": "Dose Units",
  "vr": "CS",
  "value": "GY"
}
...(truncated)...

=== Found DoseType ===
{
  "tag": "(3004,0004)",
  "keyword": "DoseType",
  "name": "Dose Type",
  "vr": "CS",
  "value": "PHYSICAL"
}
...(truncated)...

